In [1]:
pip install PyPDF2 pdfplumber pytesseract pillow pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pdf2image


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pdfplumber
import pandas as pd
import pytesseract
from pdf2image import convert_from_path


In [4]:
def extract_tables_from_pdf(pdf_path, page_number):
    tables_data = []
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number - 1]  
        tables = page.extract_tables()
        for table in tables:
            df = pd.DataFrame(table)  # DataFrame
            tables_data.append(df)
    return tables_data

In [5]:

import pdfplumber
import pandas as pd
from pdf2image import convert_from_path
import pytesseract

# extract tabls frm PDF page
def extract_tables_from_pdf(pdf_path, page_number):
    """Extract tabls  from a specifc PDF pg."""
    with pdfplumber.open(pdf_path) as pdf:
        tables = pdf.pages[page_number - 1].extract_tables()
    return [pd.DataFrame(table) for table in tables]



In [6]:

# Function to perform OCR on a given PDF page
def perform_ocr_on_page(pdf_path, page_number):
    """Perform OCR on a specific PDF pg and return extracted text."""
    images = convert_from_path(pdf_path, first_page=page_number, last_page=page_number)
    return "\n".join([pytesseract.image_to_string(image) for image in images])

# Dictionary containing PDF filenames and their  page numbers 
pdf_files = {
    "cardio_structured.pdf": 6,
    "prot_sap_102.pdf": 50,
    "prot_sap_1.pdf": 14
}

# 

In [7]:
# output Excel file
output_file = "extracted_tables.xlsx"
writer = pd.ExcelWriter(output_file, engine='openpyxl')

In [8]:
#Iterateing
for pdf_name, page_number in pdf_files.items():
    print(f"Processing {pdf_name} - Page {page_number}...")
    pdf_path = f"./{pdf_name}"
    
    tables = extract_tables_from_pdf(pdf_path, page_number)
    
    if tables:  # If  found, save them to the Excel file
        for i, table in enumerate(tables):
            table.to_excel(writer, sheet_name=f"{pdf_name}_Page{page_number}_Table{i+1}", index=False)
    else:  # If no tables are found, perform OCR and save extracted text
        extracted_text = perform_ocr_on_page(pdf_path, page_number)
        pd.DataFrame([extracted_text.split("\n")]).to_excel(writer, sheet_name=f"{pdf_name}_Page{page_number}_OCR", index=False)

Processing cardio_structured.pdf - Page 6...


c:\code files\anaconda\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Processing prot_sap_102.pdf - Page 50...
Processing prot_sap_1.pdf - Page 14...


In [9]:
# Save and close the Excel writer
writer.close()
print(f"Extraction complete. Saved to {output_file}.")


Extraction complete. Saved to extracted_tables.xlsx.
